In [ ]:
#####################
# Check if we have  #
# access to the GPU #
#####################

from IPython.display import clear_output

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
FOLDERNAME =  "University\ of\ Cyprus/MSc\ Artificial\ Intelligence/NLP-Project"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/
!ls

Mounted at /content/drive
/content/drive/My Drive/University of Cyprus/MSc Artificial Intelligence/NLP-Project
cardiffnlp	    Finetune.ipynb  lstm_model.py  preprocessing.py
data		    finetune.py     Main.ipynb	   __pycache__
DataAnalysis.ipynb  GPT-2.ipynb     main.py	   slangdict.pickle


In [ ]:
!pip install torch transformers wandb datasets ekphrasis -qqq

clear_output()

In [ ]:
from preprocessing import *
#from finetune import *

import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.9/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.9/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...


Reading twitter - 1grams ...
profit - taking hits nikkei nt <url> <url> rt <user>
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tweets_df = read_data("data/tweets_company.csv")
tweets_df.head()

,Unnamed: 0,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol
0,0,550441509175443456,VisualStockRSRC,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL
1,1,550441672312512512,KeralaGuy77,2015-01-01,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL
2,2,550441732014223360,DozenStocks,2015-01-01,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,AMZN
3,3,550442977802207232,ShowDreamCar,2015-01-01,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,TSLA
4,4,550443807834402816,i_Know_First,2015-01-01,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL


In [ ]:
msft_tweets_df = tweets_df[tweets_df['ticker_symbol'] =='MSFT']  
msft_tweets_df.head()

,Unnamed: 0,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol
19,19,550444112328261632,GetAOM,2015-01-01,$UNP $ORCL $QCOM $MSFT $AAPL Top scoring mega ...,0,0,0,MSFT
43,43,550455355134578688,BoilerWarrior,2015-01-01,"http://StockAviator.com....Top penny stocks, N...",0,0,0,MSFT
46,46,550455384734183424,StocksThatDoubl,2015-01-01,Want alerts for momentum stocks about to run? ...,0,0,0,MSFT
75,75,550466945061908482,The_Dumb_Money,2015-01-01,"IMHO, $MSFT shockingly remains MILDLY undervalued",1,0,0,MSFT
82,82,550468859879505921,UnderGradStocks,2015-01-01,@CNNMoney I like $MSFT with him in charge. He ...,0,0,0,MSFT


<h2> Set up the sentiment you want to generate </h2>

In [ ]:
sentiment = 2 #"positve" #"negative"

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv

In [ ]:
task='sentiment-financial'
#task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}/checkpoint-64"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Preprocess text (username and link placeholders)
def preprocess_tweet(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
model =  AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

text = "Good night 😊"
text = preprocess_tweet(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

array([0.01047123, 0.04755662, 0.9419722 ], dtype=float32)

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
def predict_sentiment(text):
    #scores = predict_sentiment_probs(text)
    text.to(device)
    
    output = model(**text)
    scores = output[0][0].detach().cpu().numpy()
    scores = softmax(scores)
    return  np.argmax(scores)



def tokenized(text):
    text = preprocess_tweet(text)
    encoded_input = tokenizer(text, return_tensors='pt', )
    return encoded_input

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
mask = (msft_tweets_df['post_date'] >= "2018-01-01") & (msft_tweets_df['post_date'] <= "2018-12-01")


msft_tweets_df = msft_tweets_df[mask]

In [ ]:
msft_tweets_df['tokenized'] = msft_tweets_df['body'].map(lambda x: tokenized(x))


msft_tweets_df['sentiment'] =  msft_tweets_df['tokenized'].map(lambda x: predict_sentiment(x))


In [ ]:
msft_tweets_df.head()

,Unnamed: 0,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,tokenized,sentiment
2488984,2516905,947625869954174976,permabear_uk,2018-01-01,"Microsoft $MSFT settled +1.6% at $85.54, makin...",0,0,5,MSFT,"[input_ids, attention_mask]",2
2489026,2516949,947640317712281600,MobilePotpurri,2018-01-01,True. 2017 was a big year for #AI - $IBM $GOOG...,0,2,0,MSFT,"[input_ids, attention_mask]",2
2489056,2516979,947654878402351104,OptionAlarm,2018-01-01,"With past performance like this, how can you n...",0,0,0,MSFT,"[input_ids, attention_mask]",0
2489073,2516998,947663724462137350,ExactOptionPick,2018-01-01,251 Profit-Packed Option Trades Yearly--Delive...,0,0,0,MSFT,"[input_ids, attention_mask]",2
2489086,2517011,947673358661021696,twt05,2018-01-01,"Top story: Permabear Doomster on Twitter: ""Mic...",0,0,0,MSFT,"[input_ids, attention_mask]",2


In [ ]:
msft_tweets_senti_df = msft_tweets_df[msft_tweets_df['sentiment']== sentiment]

In [ ]:
msft_tweets_senti_df.shape

(35195, 11)

In [ ]:
msft_tweets_senti_df.head()

,Unnamed: 0,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,tokenized,sentiment
2488984,2516905,947625869954174976,permabear_uk,2018-01-01,"Microsoft $MSFT settled +1.6% at $85.54, makin...",0,0,5,MSFT,"[input_ids, attention_mask]",2
2489026,2516949,947640317712281600,MobilePotpurri,2018-01-01,True. 2017 was a big year for #AI - $IBM $GOOG...,0,2,0,MSFT,"[input_ids, attention_mask]",2
2489073,2516998,947663724462137350,ExactOptionPick,2018-01-01,251 Profit-Packed Option Trades Yearly--Delive...,0,0,0,MSFT,"[input_ids, attention_mask]",2
2489086,2517011,947673358661021696,twt05,2018-01-01,"Top story: Permabear Doomster on Twitter: ""Mic...",0,0,0,MSFT,"[input_ids, attention_mask]",2
2489092,2517017,947674847294959616,SpiderMan_3s,2018-01-01,$MSFT - finally cross $100 in 2018 #2018 #Pred...,0,0,0,MSFT,"[input_ids, attention_mask]",2


In [ ]:
def boring_tweet(tweet):
    """Check if this is a boring tweet"""

    boring_stuff     = ['http', '@', '#']
    not_boring_words = len([None for w in tweet.split() if all(bs not in w.lower() for bs in boring_stuff)])

    return not_boring_words < 3

full_tweets=[]

for index, row in msft_tweets_senti_df[['body']].iterrows():
    tweet = row['body']
    if not boring_tweet(tweet):
       full_tweets.append(tweet)
    # do something with each row

len(full_tweets) 

34449

In [ ]:
total_tweet =  '<|endoftext|>' + '<|endoftext|>'.join(full_tweets) + '<|endoftext|>'



In [ ]:
file = open('positive_tweet.txt', 'w')

file.write(total_tweet)
file.close()

In [ ]:
allow_new_lines = False     
learning_rate   = 1.372e-4
epochs          = 4

In [ ]:
import transformers

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup
)

In [ ]:
import pathlib
 
tokenizer = AutoTokenizer.from_pretrained('gpt2')

model     = AutoModelForCausalLM.from_pretrained('gpt2', cache_dir=pathlib.Path('cache').resolve())

In [ ]:
handle = "msft-positive"

In [ ]:
import random

block_size    = tokenizer.model_max_length
train_dataset = TextDataset(tokenizer=tokenizer, file_path=f"positive_tweet.txt", block_size=block_size, overwrite_cache=True,)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

seed = random.randint(0,2**32-1)

training_args = TrainingArguments(
    output_dir                  = f"output/{handle}",
    overwrite_output_dir        = True,
    do_train                    = True,
    num_train_epochs            = 5,
    per_device_train_batch_size = 1,
    prediction_loss_only        = True,
    logging_steps               = 5,
    save_steps                  = 0,
    seed                        = seed,
    learning_rate               = learning_rate
)

Token indices sequence length is longer than the specified maximum sequence length for this model (2095836 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
combined_dict = {**model.config.to_dict(), **training_args.to_sanitized_dict()}

In [ ]:
trainer = Trainer(
  model         = model,
  tokenizer     = tokenizer,
  args          = training_args,
  data_collator = data_collator,
  train_dataset = train_dataset
)

In [ ]:
train_dataloader = trainer.get_train_dataloader()
num_train_steps  = len(train_dataloader)

trainer.create_optimizer_and_scheduler(num_train_steps)

trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps   = 0,
    num_training_steps = num_train_steps
)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
5,4.027700
10,3.586400
15,3.382800
20,3.324000
25,3.259500
30,3.372700
35,3.040300
40,3.516500
45,3.345000
50,3.070100


KeyboardInterrupt: ignored

In [ ]:
trainer.model.config.task_specific_params['text-generation'] = {
  'do_sample':   True,
  'min_length':  10,
  'max_length':  160,
  'temperature': 1.0,
  'top_p':       0.95,
  'prefix':      '<|endoftext|>'
}

<h2> Tweet Generation </h2>

In [ ]:
start = 'Microsoft is'

In [ ]:
start_with_bos = '<|endoftext|>' + start

encoded_prompt = trainer.tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)

In [ ]:
encoded_prompt

tensor([[50256, 15905,   318]], device='cuda:0')

In [ ]:
output_sequences = trainer.model.generate(
  input_ids            = encoded_prompt,
  max_length           = 160,              # The maximum length the generated tokens can have.
  min_length           = 10,               # The minimum length of the sequence to be generated.
  temperature          = 1.0,              # The value used to modulate the next token probabilities.
  
  top_p                = 0.95,             # If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
  do_sample            = True,             # Whether or not to use sampling ; use greedy decoding otherwise.
  num_return_sequences = 10
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated_sequences = []
predictions         = []

for generated_sequence_idx, generated_sequence in enumerate(output_sequences):

    generated_sequence = generated_sequence.tolist()
  
    text = trainer.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
  
    if not allow_new_lines:
        limit = text.find('\n')
        text  = text[: limit if limit != -1 else None]

    generated_sequences.append(text.strip())

for i, g in enumerate(generated_sequences): predictions.append([start, g])

In [ ]:
for s in generated_sequences: print(s)

Microsoft is buying more Xbox to deliver a more-innovative platform, and how that looks in Azure.
Microsoft is about to see a massive growth in enterprise software.
Microsoft is to buy 4X shares to help it achieve its own market cap $MSFT $SNAP
Microsoft is looking to make use of its cloud tech to provide cloud and app apps to small business customers. Microsoft is an established #cloud leader in enterprise cloud tech, which is the tech sector that we most need. https://stocknews.com/news/microsoft-is-looking-to-make-use-of-its-cloud-tech-as-app-cloud-solutions/…
Microsoft is building Cortana for Microsoft. And Cortana? $MSFT
Microsoft is planning a $100 billion sales deal that would be called "The One. Very Soon".
Microsoft is making huge strides in its $MSFT. See it below #technology
Microsoft is making big moves to rival Amazon. It's holding a "lot of the tech assets, like its cloud, which will be sold off $MSFT. $FUSZ is also doing $BABABA for Azure.
Microsoft is reportedly "one of

In [ ]:
generated_sequences = generated_sequences[:-2]

In [ ]:
for s in generated_sequences: print(s)

Microsoft is buying more Xbox to deliver a more-innovative platform, and how that looks in Azure.
Microsoft is about to see a massive growth in enterprise software.
Microsoft is to buy 4X shares to help it achieve its own market cap $MSFT $SNAP
Microsoft is looking to make use of its cloud tech to provide cloud and app apps to small business customers. Microsoft is an established #cloud leader in enterprise cloud tech, which is the tech sector that we most need. https://stocknews.com/news/microsoft-is-looking-to-make-use-of-its-cloud-tech-as-app-cloud-solutions/…
Microsoft is building Cortana for Microsoft. And Cortana? $MSFT
Microsoft is planning a $100 billion sales deal that would be called "The One. Very Soon".
Microsoft is making huge strides in its $MSFT. See it below #technology
Microsoft is making big moves to rival Amazon. It's holding a "lot of the tech assets, like its cloud, which will be sold off $MSFT. $FUSZ is also doing $BABABA for Azure.
